In [3]:
from tmc_summarizer import TMC_File 
tmc = TMC_File('/Volumes/GoogleDrive/My Drive/tmc_copy/1_Central_Ave.xls')

Reading 1_Central_Ave.xls


In [4]:
# tmc.peak_hour_factor('am')
tmc.peak_hour_factor('am')

,EB U,EB Left,EB Thru,EB Right,EB Xwalk Xings,WB U,WB Left,WB Thru,WB Right,WB Xwalk Xings,...,NB Thru,NB Right,NB Xwalk Xings,SB U,SB Left,SB Thru,SB Right,SB Xwalk Xings,total_15_min,total_hourly
datetime,,,,,,,,,,,,,,,,,,,,,
2022-03-24 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-24 00:15:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-24 00:30:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-24 00:45:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-24 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-24 01:15:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-24 01:30:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-24 01:45:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-24 02:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
